In [1]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

from sklearn.preprocessing import StandardScaler
import sklearn

from utils import plot_projections

from scipy.stats import kurtosis, skew

%matplotlib widget

In [2]:
laser_dir = '/home/hashim/PHD/audio_data/Kareem/Laser/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files.sort()
print(laser_files)
print(len(laser_files))

['/home/hashim/PHD/audio_data/Kareem/Laser/laser_000.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_001.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_002.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_003.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_004.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_005.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_006.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_007.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_008.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_009.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_010.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_011.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_012.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_013.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_014.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_015.mp3', '/home/hashim/PHD/audio_data/Kareem/Laser/laser_016.mp3', '/home/hashim

In [3]:
orig_dir = '/home/hashim/PHD/audio_data/Kareem/Original/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files.sort()
print(orig_files)
print(len(orig_files))

['/home/hashim/PHD/audio_data/Kareem/Original/original_000.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_001.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_002.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_003.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_004.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_005.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_006.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_007.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_008.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_009.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_010.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_011.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_012.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_013.mp3', '/home/hashim/PHD/audio_data/Kareem/Original/original_014.mp3', '/home/hashim/PHD/audio_data/Kareem/Ori

In [4]:
db1 = pywt.Wavelet('db1')

## Laser

In [5]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
laser_skew_feat = []
laser_kurt_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
    
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
    laser_skew_feat.append(skew_feat)
    laser_kurt_feat.append(kurt_feat)    
    
laser_sigma_feat = np.array(laser_sigma_feat)
laser_skew_feat = np.array(laser_skew_feat)
laser_kurt_feat = np.array(laser_kurt_feat)

In [6]:
print(laser_sigma_feat.shape)
print(laser_skew_feat.shape)
print(laser_kurt_feat.shape)

(28, 6)
(28, 6)
(28, 6)


## Original

In [7]:
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
orig_skew_feat = []
orig_kurt_feat = []

for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    orig_sigma_feat.append(sigma_feat)
    orig_skew_feat.append(skew_feat)
    orig_kurt_feat.append(kurt_feat)    
    
orig_sigma_feat = np.array(orig_sigma_feat)
orig_skew_feat = np.array(orig_skew_feat)
orig_kurt_feat = np.array(orig_kurt_feat)

In [8]:
# cluster plotting function for sigma, skew and kurtosis

def plot_clusters_skk(band_num):
    
    plt.figure(figsize=(10,3))
    
#     sigma vs skew
    plt.subplot(1, 3, 1)
    plt.scatter(orig_sigma_feat[:,band_num], orig_skew_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_sigma_feat[:,band_num], laser_skew_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("variance")
    plt.ylabel("skew")
    plt.legend()
    
#     skew vs kurtosis
    plt.subplot(1, 3, 2)
    plt.scatter(orig_skew_feat[:,band_num], orig_kurt_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_skew_feat[:,band_num], laser_kurt_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("skew")
    plt.ylabel("kurtosis")
    plt.legend()
     
#     kurtosis vs sigma
    plt.subplot(1, 3, 3)
    plt.scatter(orig_kurt_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_kurt_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("kurtosis")
    plt.ylabel("variance")
    plt.legend()
    
    plt.suptitle("Laser vs Original for {}th Band".format(band_num))
    
    fig = plt.figure(figsize=(12,6))
#     plt.figure(figsize=(20,6))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(orig_sigma_feat[:,band_num], orig_skew_feat[:,band_num], orig_kurt_feat[:,band_num], marker='o', label="Original")
    ax.scatter(laser_sigma_feat[:,band_num], laser_skew_feat[:,band_num], laser_kurt_feat[:,band_num], marker='x', label="Laser")
    ax.set_xlabel('variance')
    ax.set_ylabel('skew')
    ax.set_zlabel('kurtosis')
    ax.legend()

In [9]:
aprox_detail_coeff = 0 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
aprox_detail_coeff = 1 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
aprox_detail_coeff = 2 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
aprox_detail_coeff = 3 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
aprox_detail_coeff = 4 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
aprox_detail_coeff = 5 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# SVM with Linear Kernel

In [28]:
import sklearn

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

In [29]:
X_laser = np.concatenate((laser_sigma_feat, laser_skew_feat, laser_kurt_feat), axis=1)
laser_label = np.array(['laser']*X_laser.shape[0])
print(laser_label)
print(laser_label.shape)
X_laser.shape

['laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser' 'laser'
 'laser']
(28,)


(28, 18)

In [30]:
X_orig = np.concatenate((orig_sigma_feat, orig_skew_feat, orig_kurt_feat), axis=1)
orig_label = np.array(['original']*X_orig.shape[0])
print(orig_label)
print(orig_label.shape)
X_orig.shape

['original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original' 'original' 'original'
 'original' 'original' 'original' 'original']
(28,)


(28, 18)

In [31]:
clf = make_pipeline(StandardScaler(), LinearSVC())

In [32]:
X = np.concatenate((X_laser, X_orig))
X.shape

(56, 18)

In [33]:
y = np.concatenate((laser_label, orig_label))
y.shape

(56,)

In [34]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [36]:
X_train.shape

(37, 18)

In [37]:
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC())])

In [38]:
y_pred = clf.predict(X_test)

In [39]:
print("accuracy = ", accuracy_score(y_test, y_pred))

accuracy =  1.0


In [40]:
cm = confusion_matrix(y_test, y_pred)

In [41]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# with Log sigma

In [42]:
import math

In [43]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

laser_sigma_feat = []
laser_skew_feat = []
laser_kurt_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf
        
        sigma_feat.append(np.log(np.var(cf)))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    laser_sigma_feat.append(sigma_feat)
    laser_skew_feat.append(skew_feat)
    laser_kurt_feat.append(kurt_feat)    
    
laser_sigma_feat = np.array(laser_sigma_feat)
laser_skew_feat = np.array(laser_skew_feat)
laser_kurt_feat = np.array(laser_kurt_feat)

In [44]:
print(laser_sigma_feat.shape)
print(laser_skew_feat.shape)
print(laser_kurt_feat.shape)

(28, 6)
(28, 6)
(28, 6)


In [45]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

orig_sigma_feat = []
orig_skew_feat = []
orig_kurt_feat = []

for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf
        
        sigma_feat.append(np.log(np.var(cf)))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    orig_sigma_feat.append(sigma_feat)
    orig_skew_feat.append(skew_feat)
    orig_kurt_feat.append(kurt_feat)    
    
orig_sigma_feat = np.array(orig_sigma_feat)
orig_skew_feat = np.array(orig_skew_feat)
orig_kurt_feat = np.array(orig_kurt_feat)

In [46]:
aprox_detail_coeff = 0 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
aprox_detail_coeff = 1 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
aprox_detail_coeff = 2 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
aprox_detail_coeff = 3 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_7828/2478823229.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(12,6))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [50]:
aprox_detail_coeff = 4 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
aprox_detail_coeff = 5 
plot_clusters_skk(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …